# import libraries 

In [17]:
# Seaborn for good visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
# For hyperparameter tuning
from sklearn.preprocessing import StandardScaler  # For normalization
from sklearn.feature_selection import SelectKBest, f_classif  # Feature selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, confusion_matrix, roc_curve, auc


In [18]:
data = load_breast_cancer()
df = pd.DataFrame(data.data , columns = data.feature_names)
df['tareget'] = data.target
df.head()


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,tareget
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [19]:
X = data.data
y = data.target

## split data

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42, stratify = y )



print("Training Sample: ", X_train.shape[0])
print("Testing Sample: ", X_test.shape[0])


Training Sample:  455
Testing Sample:  114


In [20]:
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)


In [27]:
selector = SelectKBest(score_func=f_classif , k=20)
X_train = selector.fit_transform(X_train , y_train)
X_test = selector.transform(X_test)
print(f"selected Features Shape : {X_train.shape}")

selected Features Shape : (455, 20)


In [38]:
log_reg = LogisticRegression(max_iter = 5000 , random_state = 42)
## HyperPaprameter grid
param_grid = {
    'C': [0.0001,0.01,1,12,50,100],
    'penalty':['l1','l2'],
    'solver':['liblinear','saga']
}

# get best combination for model training using GridSearchCV
grid_search = GridSearchCV(log_reg , param_grid, cv = 5, scoring = 'accuracy', n_jobs = -1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

print(f"best parameters :{grid_search.best_params_}")

best parameters :{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}


In [41]:
# # Cross- Validation accuracy
cv_scores = cross_val_score(best_model, X_train, y_train, cv = 5)
## Prediction
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)
recall =recall_score(y_test,y_pred)
f1 = fbeta_score(y_test,y_pred , beta =1)

print(f"accuracy:{accuracy*100:.2f}%")
print(f"precision:{precision:.2f}")
print(f"recall:{recall:.2f}%")
print(f"f-1_score:{f1:.2f}%")



accuracy:97.37%
precision:0.99
recall:0.97%
f-1_score:0.98%
